# 13. Fixing calibration parameters
In this notebook we will demonstrate how to fix the calibration parameters $\gamma$ and $\alpha$. This can be useful in setups where you have insufficient reference sections to calibrate these, but you do have information on these parameters from previous setups with the same fiber.

We will be using the same dataset as notebook 5. Calibration of single-ended measurement with OLS

In [ ]:
import os

from dtscalibration import read_silixa_files
from dtscalibration.dts_accessor import DtsAccessor  # noqa: E402
from dtscalibration.variance_stokes import variance_stokes_constant
import matplotlib.pyplot as plt

%matplotlib inline

filepath = os.path.join("..", "..", "tests", "data", "single_ended")

ds = read_silixa_files(directory=filepath, timezone_netcdf="UTC", file_ext="*.xml")

ds100 = ds.sel(x=slice(-30, 101))  # only calibrate parts of the fiber, in meters
sections = {
    "probe1Temperature": [
        slice(20, 25.5)
    ],  # we only use the warm bath in this notebook
}

From the previous calibration we know that the $\gamma$ parameter value was 481.9 and the $\alpha$ value was -2.014e-05.
We define these, along with their variance. In this case we do not know what the variance was, as we ran an OLS calibration, so we will set the variance to 0.

It is important to note that when setting parameters, the covariances between the parameters are not taken into account in the uncertainty.

In [ ]:
fix_gamma = (481.9, 0)  # (gamma value, gamma variance)
fix_dalpha = (-2.014e-5, 0)  # (alpha value, alpha variance)

st_var, resid = variance_stokes_constant(
    ds.dts.st, sections, ds.dts.acquisitiontime_fw, reshape_residuals=True
)
ast_var, _ = variance_stokes_constant(
    ds.dts.ast, sections, ds.dts.acquisitiontime_fw, reshape_residuals=False
)
out = ds100.dts.calibrate_single_ended(
    sections=sections,
    st_var=st_var,
    ast_var=ast_var,
    fix_gamma=fix_gamma,
    fix_dalpha=fix_dalpha,
)

Let's see if fixing the parameters worked:

In [ ]:
print("gamma used in calibration:", out.gamma.values)
print("dalpha used in calibration:", out.dalpha.values)

Let's plot the calibrated temperature. You'll see that this gives the same result as in notebook 05.

In [ ]:
out.isel(time=0).tmpf.plot(
    linewidth=1, figsize=(12, 8), label="User calibrated"
)  # plot the temperature calibrated by us
ds100.isel(time=0).tmp.plot(
    linewidth=1, label="Device calibrated"
)  # plot the temperature calibrated by the device
plt.title("Temperature at the first time step")
plt.legend()